In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [3]:
import pickle

In [4]:
with open('../model/ada_lg_model.pkl', 'rb') as pick:
    model_ada_lg = pickle.load(pick)


In [5]:
drugs= pd.read_csv("../data/processed/drugs_desc.csv")
drugs_rdkfp = pd.read_csv("../data/processed/drugs_rdkfp.csv")

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [8]:
X= drugs[["tpsa","mol_w","qed","HBA","HBD","logP","MR"]]

In [9]:
joined_X= X.join(drugs_rdkfp)

In [11]:
scaler= MinMaxScaler()

In [12]:
X_scaled= scaler.fit_transform(joined_X)

In [15]:
preds= model_ada_lg.predict(X_scaled)

In [20]:
drugs_info = pd.read_csv("../data/raw/repurposing_samples_20180907.txt", sep="\t")

In [21]:
drugs_info.columns

Index(['broad_id', 'pert_iname', 'qc_incompatible', 'purity', 'vendor',
       'catalog_no', 'vendor_name', 'expected_mass', 'smiles', 'InChIKey',
       'pubchem_cid', 'deprecated_broad_id'],
      dtype='object')

In [26]:
preds_df = drugs_info[["pert_iname", "smiles"]]

In [28]:
preds_df= preds_df.dropna()

In [29]:
preds_df["predictions"] = preds

In [30]:
preds_df

,pert_iname,smiles,predictions
0,"[sar9,met(o2)11]-substance-p",CC(C)C[C@H](NC(=O)CN(C)C(=O)[C@H](Cc1ccccc1)NC...,0
1,"1-((Z)-3-Chloroallyl)-1,3,5,7-tetraazaadamanta...",Cl\C=C/C[N+]12CN3CN(CN(C3)C1)C2,0
2,"1-(1,2-Diphenylethyl)piperidine-(+/-)",C(C(N1CCCCC1)c1ccccc1)c1ccccc1,0
3,1-(2-chloro-5-methylphenoxy)-3-(isopropylamino...,CC(C)NCC(O)COc1cc(C)ccc1Cl,0
4,1-acetyl-4-methylpiperazine,CN1CCN(CC1)C(C)=O,0
...,...,...,...
10142,zotepine,CN(C)CCOC1=Cc2ccccc2Sc2ccc(Cl)cc12,0
10143,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,0
10144,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,0
10145,ZSTK-474,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,0


In [ ]:
#I didn't realize earlier that there are some repeated compounds. Probably due to some compounds
#having multiple vendors

In [53]:
preds_df["smiles"].value_counts()

CCCC(CCC)C(O)=O                                                                                  8
CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=O)N2[C@H]1C(O)=O                                   8
CC(/C=C/C1=C(C)CCCC1(C)C)=C\C=C\C(C)=C\C(O)=O                                                    7
CN(C)[C@H]1[C@@H]2[C@@H](O)[C@H]3C(C(=O)c4c(O)cccc4[C@@]3(C)O)C(=O)[C@]2(O)C(=O)C(C(N)=O)C1=O    7
COc1ccccc1N1CCN(CC(O)COc2cccc3ccccc23)CC1                                                        7
                                                                                                ..
CCn1cc(c(n1)-c1ccc(NC(=O)N(C)C)cc1)-c1ccnc2[nH]c(cc12)-c1cccc(CN(C)C)c1                          1
C1[C@@H](NC2CCNCC2)[C@@H]1c1ccccc1                                                               1
CCOC(=O)CCNc1cc(nc(n1)-c1cccnc1)N1CCc2ccccc2CC1                                                  1
COc1ccc(cc1)S(=O)(=O)Nc1c(C)cc(C)cc1C                                                            1
OCCN1CCN(C

In [60]:
preds_df= preds_df.drop_duplicates(subset=["pert_iname"], keep="first")

In [62]:
mask = preds_df["predictions"] == 1

In [63]:
preds_df[mask]

,pert_iname,smiles,predictions
5,1-azakenpaullone,Brc1ccc2[nH]c-3c(CC(=O)Nc4cccnc-34)c2c1,1
51,2-chloroadenosine,Nc1nc(Cl)nc2n(cnc12)[C@@H]1O[C@H](CO)[C@@H](O)...,1
87,2-TEDC,Oc1ccc(\C=C(/C#N)C(=O)OCCc2cccs2)cc1O,1
133,"3,5-DHPG-(S)",N[C@H](C(O)=O)c1cc(O)cc(O)c1,1
145,4-carboxy-3-hydroxyphenylglycine-(RS),NC(C(O)=O)c1ccc(C(O)=O)c(O)c1,1
...,...,...,...
9897,VX-222,C[C@H]1CC[C@@H](CC1)C(=O)N([C@@H]1CC[C@H](O)CC...,1
9926,WDR5-0103,COC(=O)c1ccc(N2CCN(C)CC2)c(NC(=O)c2cccc(OC)c2)c1,1
10042,zalcitabine,Nc1ccn([C@H]2CC[C@@H](CO)O2)c(=O)n1,1
10078,zidovudine,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,1


In [34]:
with open('../model/simple_deep_l_model.pkl', 'rb') as picky:
    model_deep_l = pickle.load(picky)

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-12-28 10:13:00         3299
metadata.json                                  2022-12-28 10:13:00           64
variables.h5                                   2022-12-28 10:13:00      6361328
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dropout
......vars
...metrics\auc
......vars
.........0
.........1
.........2
.........3
...metrics\false_negatives
......vars
.........0
...metrics\false_positives
......vars
.........0
...metrics\mean
......vars
.........0
.........1
...metrics\recall
......vars
.........0
.........1
...metrics\true_positives
......vars
.........0
...optimizer
......vars
.........0
.........1
.........2
.........3
.........4
.........5

In [36]:
deep_l_preds= model_deep_l.predict(X_scaled)

317/317 [==============================] - 1s 3ms/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [66]:
deep_df= drugs_info[["pert_iname", "smiles"]].copy()

In [67]:
deep_df= deep_df.dropna()

In [69]:
deep_df["predictions"] = deep_l_preds

In [70]:
deep_df

,pert_iname,smiles,predictions
0,"[sar9,met(o2)11]-substance-p",CC(C)C[C@H](NC(=O)CN(C)C(=O)[C@H](Cc1ccccc1)NC...,1.013333e-01
1,"1-((Z)-3-Chloroallyl)-1,3,5,7-tetraazaadamanta...",Cl\C=C/C[N+]12CN3CN(CN(C3)C1)C2,3.920485e-06
2,"1-(1,2-Diphenylethyl)piperidine-(+/-)",C(C(N1CCCCC1)c1ccccc1)c1ccccc1,6.339062e-08
3,1-(2-chloro-5-methylphenoxy)-3-(isopropylamino...,CC(C)NCC(O)COc1cc(C)ccc1Cl,4.321629e-08
4,1-acetyl-4-methylpiperazine,CN1CCN(CC1)C(C)=O,5.793416e-05
...,...,...,...
10142,zotepine,CN(C)CCOC1=Cc2ccccc2Sc2ccc(Cl)cc12,6.770114e-01
10143,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,1.095973e-03
10144,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,1.095973e-03
10145,ZSTK-474,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,9.961943e-06


In [71]:
deep_df= deep_df.drop_duplicates(subset=["pert_iname"], keep="first")

In [72]:
deep_df

,pert_iname,smiles,predictions
0,"[sar9,met(o2)11]-substance-p",CC(C)C[C@H](NC(=O)CN(C)C(=O)[C@H](Cc1ccccc1)NC...,1.013333e-01
1,"1-((Z)-3-Chloroallyl)-1,3,5,7-tetraazaadamanta...",Cl\C=C/C[N+]12CN3CN(CN(C3)C1)C2,3.920485e-06
2,"1-(1,2-Diphenylethyl)piperidine-(+/-)",C(C(N1CCCCC1)c1ccccc1)c1ccccc1,6.339062e-08
3,1-(2-chloro-5-methylphenoxy)-3-(isopropylamino...,CC(C)NCC(O)COc1cc(C)ccc1Cl,4.321629e-08
4,1-acetyl-4-methylpiperazine,CN1CCN(CC1)C(C)=O,5.793416e-05
...,...,...,...
10141,zotarolimus,CO[C@@H]1C[C@H](C[C@@H](C)[C@@H]2CC(=O)[C@H](C...,6.532036e-01
10142,zotepine,CN(C)CCOC1=Cc2ccccc2Sc2ccc(Cl)cc12,6.770114e-01
10143,zoxazolamine,Nc1nc2cc(Cl)ccc2o1,1.095973e-03
10145,ZSTK-474,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,9.961943e-06


In [79]:
mask90 = deep_df["predictions"]>0.9

In [80]:
deep_df[mask90]

,pert_iname,smiles,predictions
34,12-O-tetradecanoylphorbol-13-acetate,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,0.982217
324,AC-710,CCN1C(C)(C)CC(CC1(C)C)Oc1ccc(nc1)C(=O)Nc1ccc(N...,0.927637
368,acetriazoic-acid,CC(=O)Nc1c(I)cc(I)c(C(O)=O)c1I,0.933865
871,apraclonidine,Nc1cc(Cl)c(NC2=NCCN2)c(Cl)c1,0.907846
1053,AVE-0991,CCNC(=O)NS(=O)(=O)c1sc(CC(C)C)cc1-c1ccc(Cn2c(C...,0.924707
...,...,...,...
9649,UK-356618,C[C@@H](NC(=O)[C@@H](NC(=O)[C@H](CCCc1ccc(c(C)...,0.970714
9866,vortioxetine,Cc1ccc(Sc2ccccc2N2CCNCC2)c(C)c1,0.941291
9926,WDR5-0103,COC(=O)c1ccc(N2CCN(C)CC2)c(NC(=O)c2cccc(OC)c2)c1,0.924557
10042,zalcitabine,Nc1ccn([C@H]2CC[C@@H](CO)O2)c(=O)n1,0.958454


In [75]:
mask50 = deep_df["predictions"]>0.5

In [76]:
deep_df[mask50]

,pert_iname,smiles,predictions
24,10-deacetylbaccatin,CC(=O)O[C@@]12CO[C@@H]1C[C@H](O)[C@]1(C)[C@@H]...,0.750009
34,12-O-tetradecanoylphorbol-13-acetate,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,0.982217
49,2-chloro-N6-cyclopentyladenosine,OC[C@H]1O[C@H]([C@H](O)[C@@H]1O)n1cnc2c(NC3CCC...,0.840763
51,2-chloroadenosine,Nc1nc(Cl)nc2n(cnc12)[C@@H]1O[C@H](CO)[C@@H](O)...,0.704971
97,2'-c-methylguanosine,C[C@@]1(O)[C@H](O)[C@@H](CO)O[C@H]1n1cnc2c(O)n...,0.534111
...,...,...,...
10044,zaldaride,CC1(CN2CCC(CC2)n2c3ccccc3[nH]c2=O)OCc2ccccc2-n...,0.563703
10074,zebularine,OC[C@H]1O[C@H]([C@H](O)[C@@H]1O)n1cccnc1=O,0.548952
10078,zidovudine,Cc1cn([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)c(...,0.998899
10141,zotarolimus,CO[C@@H]1C[C@H](C[C@@H](C)[C@@H]2CC(=O)[C@H](C...,0.653204
